In [4]:
import os
import utils

utils.load_env()
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [5]:
from langchain_core.messages import HumanMessage

# for llm model
from langchain_openai import ChatOpenAI
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from tools import find_place_from_text, nearby_search
from typing import Dict, List, Tuple
from langchain.agents import (
    AgentExecutor,
)
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Bind the tools to the model
tools = [find_place_from_text, nearby_search]  # Include both tools if needed

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

meta = utils.load_agent_meta()[0]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", meta['prompt']),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)


JSONDecodeError: Invalid control character at: line 4 column 358 (char 398)

In [ ]:
def submitUserMessage(user_input: str) -> str:
    responce = agent_executor.invoke({"input": user_input, "chat_history": []})
    return responce["output"]

submitUserMessage("ค้นหาร้านกาแฟใกล้มาบุญครอง")

Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')




> Entering new AgentExecutor chain...


Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')



Invoking: `nearby_search` with `{'input_dict': {'keyword': 'coffee shop', 'location_name': 'มาบุญครอง', 'radius': 1500, 'place_type': 'cafe'}}`




Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')
Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')



        address: 991/1 Rama I Rd, Pathum Wan

        location: {'lat': 13.7458781, 'lng': 100.5341476}

        name: BEANS Coffee Roaster Paragon

        opening hours: {'open_now': True}

        rating: 5

        plus code: 7P52PGWM+9M


        
        address: 43/4 Rama VI Rd, Rong Muang, Pathum Wan

        location: {'lat': 13.7497223, 'lng': 100.521221}

        name: Olympic Coffee

        opening hours: {'open_now': True}

        rating: 4.8

        plus code: 7P52PGXC+VF


        
        address: 994 Makkasan, Ratchathewi

        location: {'lat': 13.7495947, 'lng': 100.5426623}

        name: Earthy Roaster (Specialty Coffee)

        opening hours: {'open_now': True}

        rating: 4.8

        plus code: 7P52PGXV+R3


        
        address: MINT TOWER Bantadthong Phetchaburi Rd, Subdistrict Pathum Wan

        location: {'lat': 13.7450255, 'lng': 100.5235735}

        name: Treasure specialty coffee

        opening hours: {'open_now': True}

        ratin

Unable to load requested LangChainTracer. To disable this warning, unset the LANGCHAIN_TRACING_V2 environment variables.
LangSmithUserError('API key must be provided when using hosted LangSmith API')


Here is a list of coffee shops near มาบุญครอง (MBK Center):

1. **BEANS Coffee Roaster Paragon**
   - Address: 991/1 Rama I Rd, Pathum Wan
   - Rating: 5.0
   - Open Now: Yes

2. **Olympic Coffee**
   - Address: 43/4 Rama VI Rd, Rong Muang, Pathum Wan
   - Rating: 4.8
   - Open Now: Yes

3. **Earthy Roaster (Specialty Coffee)**
   - Address: 994 Makkasan, Ratchathewi
   - Rating: 4.8
   - Open Now: Yes

4. **Treasure specialty coffee**
   - Address: MINT TOWER Bantadthong Phetchaburi Rd, Pathum Wan
   - Rating: 4.9
   - Open Now: Yes

5. **The Basic Coffee BKK**
   - Address: ตึก Gaysorn Amarin, ชั้น 2 496-502 Phloen Chit Rd, Khwaeng Lumphini, Pathum Wan
   - Rating: 4.7
   - Open Now: Yes

6. **CRANNIES Coffee & Brunch**
   - Address: 126 ซ. จุฬาลงกรณ์ 50 Wang Mai, Pathum Wan
   - Rating: 5.0
   - Open Now: Yes

7. **Bullet Thai Craft Beer**
   - Address: 85 1 Akkharanithi Alley, Thanon Phaya Thai, Ratchathewi
   - Rating: 5.0
   - Open Now: Yes

8. **Sip 'n Drip**
   - Address: 36, 6

"Here is a list of coffee shops near มาบุญครอง (MBK Center):\n\n1. **BEANS Coffee Roaster Paragon**\n   - Address: 991/1 Rama I Rd, Pathum Wan\n   - Rating: 5.0\n   - Open Now: Yes\n\n2. **Olympic Coffee**\n   - Address: 43/4 Rama VI Rd, Rong Muang, Pathum Wan\n   - Rating: 4.8\n   - Open Now: Yes\n\n3. **Earthy Roaster (Specialty Coffee)**\n   - Address: 994 Makkasan, Ratchathewi\n   - Rating: 4.8\n   - Open Now: Yes\n\n4. **Treasure specialty coffee**\n   - Address: MINT TOWER Bantadthong Phetchaburi Rd, Pathum Wan\n   - Rating: 4.9\n   - Open Now: Yes\n\n5. **The Basic Coffee BKK**\n   - Address: ตึก Gaysorn Amarin, ชั้น 2 496-502 Phloen Chit Rd, Khwaeng Lumphini, Pathum Wan\n   - Rating: 4.7\n   - Open Now: Yes\n\n6. **CRANNIES Coffee & Brunch**\n   - Address: 126 ซ. จุฬาลงกรณ์ 50 Wang Mai, Pathum Wan\n   - Rating: 5.0\n   - Open Now: Yes\n\n7. **Bullet Thai Craft Beer**\n   - Address: 85 1 Akkharanithi Alley, Thanon Phaya Thai, Ratchathewi\n   - Rating: 5.0\n   - Open Now: Yes\n\n